In [1]:
import torch
import numpy as np
import skimage
import torch.nn as nn
import torch.nn.functional as F
from models.darcy import cc1
from models.darcy import cc2
from models.darcy import bc
from utils.image_gradient import SobelFilter
from FEA_simp import ComputeTarget

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model = torch.load('experiments/codec/mixed_residual/debug/grf'\
            '_kle512_ntrain4010_run1_bs32_lr0.001_epochs5000/checkpoints/model_epoch5000.pth')

data = np.loadtxt("data/rho20x40_SIMP_Edge.txt", dtype=np.float32)
# [bs, 1, 20, 40]
data = data.reshape(-1,1,40,20).transpose([0,1,3,2])

data_u = np.loadtxt("data/dis20x40_SIMP_Edge.txt", dtype=np.float32)
data_s = np.loadtxt("data/stress20x40_SIMP_Edge.txt", dtype=np.float32)

ref_u0 = torch.from_numpy(data_u).unsqueeze(1).to(device)
ref_uy = ref_u0[:,:,range(1,1722,2)]
ref_ux = ref_u0[:,:,range(0,1722,2)]
ref_s0 = torch.from_numpy(data_s).unsqueeze(1).to(device)
ref_sx = ref_s0[:,:,range(0,2583,3)]
ref_sy = ref_s0[:,:,range(1,2583,3)]
ref_sxy = ref_s0[:,:,range(2,2583,3)]
# [bs, 5, 21, 41]
ref = torch.cat([ref_ux, ref_uy, ref_sx, ref_sy, ref_sxy],1).view(-1,5,41,21).permute(0,1,3,2)

input = torch.from_numpy(data).to(device)
output = model(input)

RuntimeError: CUDA out of memory. Tried to allocate 104.00 MiB (GPU 0; 11.91 GiB total capacity; 10.11 GiB already allocated; 85.88 MiB free; 202.67 MiB cached)

In [3]:
sobel_filter = SobelFilter(64, correct=False, device=device)
# post output
WEIGHTS_2x2 = torch.FloatTensor( np.ones([1,1,2,2])/4 ).to(device)
o0 = F.conv2d(output[:,[0]], WEIGHTS_2x2, stride=1, padding=0, bias=None)
o1 = F.conv2d(output[:,[1]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o2 = F.conv2d(output[:,[2]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o3 = F.conv2d(output[:,[3]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o4 = F.conv2d(output[:,[4]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
output_post = torch.cat([o0,o1,o2,o3,o4],1)
print(output_post.shape)

torch.Size([1385, 5, 20, 40])


In [4]:
print(cc2(output_post, sobel_filter))

tensor(2.8988, device='cuda:0', grad_fn=<MeanBackward0>)


In [5]:
print (cc1(input, output, output_post, sobel_filter, device))

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)


In [7]:
loss_boundary = bc(output, output_post)
print(loss_boundary)

tensor(0.0058, device='cuda:0', grad_fn=<MeanBackward0>)


In [11]:
# cc2 loss detail
dsxdx = sobel_filter.grad_h(output_post[:, [2]])
dsydy = sobel_filter.grad_v(output_post[:, [3]])
dsxydx = sobel_filter.grad_h(output_post[:, [4]])
dsxydy = sobel_filter.grad_v(output_post[:, [4]])

ds = torch.cat([dsxdx+dsxydy, dsydy+dsxydx],1)

In [12]:
dsxdx.shape

torch.Size([1385, 1, 20, 40])

In [33]:
print(dsxdx[0,0,:,-2])

tensor([0.2637, 0.3571, 0.3573, 0.3508, 0.3478, 0.3514, 0.3559, 0.3559, 0.3541,
        0.3537, 0.3537, 0.3543, 0.3571, 0.3593, 0.3556, 0.3472, 0.3429, 0.3464,
        0.3500, 0.2623], device='cuda:0', grad_fn=<SelectBackward>)


In [29]:
dsxdx.mean([0])

tensor([[[-2.2466e-06,  1.5562e-04,  8.0763e-05, -2.8926e-05, -4.4579e-05,
          -3.5976e-05, -3.5305e-05, -2.0696e-05, -3.0079e-06, -3.5213e-07,
          -4.9989e-06, -7.8270e-06, -7.1959e-06, -3.8826e-06, -4.1764e-06,
          -9.9766e-07,  1.0633e-05,  8.0142e-06, -3.9688e-06, -5.6745e-06,
          -4.5286e-07,  4.5850e-06,  1.9796e-06, -3.8503e-06, -4.7108e-07,
           1.5324e-06, -2.1786e-06, -8.3796e-07,  1.3151e-05,  3.4179e-05,
           5.0584e-06, -5.2194e-05,  5.8720e-05,  1.7942e-04, -2.9829e-04,
           1.9202e-04, -1.6025e-02,  5.9525e-02,  2.8058e-01, -5.9864e-02],
         [ 5.4913e-06,  2.2033e-04,  9.0546e-05, -4.5635e-05, -5.2762e-05,
          -4.0271e-05, -4.3614e-05, -3.0095e-05, -5.6838e-06, -4.4983e-06,
          -1.3149e-05, -1.2619e-05, -9.4660e-06, -7.4983e-06, -7.6170e-06,
           1.0316e-06,  1.6221e-05,  1.2138e-05, -1.9178e-06, -7.2230e-06,
          -2.4375e-06,  5.9367e-06,  4.4030e-06, -2.6833e-06, -2.5404e-06,
          -7.8459e-07, -

In [30]:
output[:, [2]].mean([0])

tensor([[[ 1.7647e-04,  4.9508e-05,  1.2522e-04,  1.2853e-04,  7.5470e-05,
           1.4630e-05,  4.2478e-06, -1.0253e-05,  4.0300e-05, -2.8367e-05,
           7.9158e-08, -1.2256e-05,  4.5822e-08, -1.5376e-05, -2.6871e-06,
          -6.1857e-06, -9.5121e-06, -2.6373e-05, -7.2126e-06, -1.5809e-05,
          -2.6649e-06, -2.2805e-05, -1.6091e-05, -3.0608e-05, -2.9315e-05,
          -3.7857e-05, -3.3421e-05, -5.3041e-05, -2.4989e-05, -6.5961e-07,
          -8.9977e-07,  1.1645e-05, -1.9943e-05, -2.0844e-04,  4.1694e-04,
           2.9118e-04, -1.4556e-03,  3.8867e-03, -5.5018e-02,  4.4071e-01,
           9.9999e-01],
         [-4.2292e-04, -4.0744e-04,  2.1095e-04,  3.3285e-04,  2.9065e-04,
           1.5911e-04,  1.4339e-04, -1.2806e-05, -2.5192e-05, -2.2246e-05,
           5.1131e-06, -3.1055e-05, -2.2675e-05, -6.2865e-05, -5.4838e-05,
          -6.5167e-05, -8.2200e-05, -3.3519e-05, -1.3123e-05, -4.5031e-05,
          -6.0090e-05, -4.8249e-05, -3.5900e-05, -1.2429e-05, -4.7977e-05,
 

In [31]:
output_post[:, [2]].mean([0])

tensor([[[-1.5109e-04, -5.4395e-06,  1.9939e-04,  2.0687e-04,  1.3497e-04,
           8.0345e-05,  3.1145e-05, -1.9877e-06, -8.8759e-06, -1.1355e-05,
          -9.5296e-06, -1.6485e-05, -2.5218e-05, -3.3941e-05, -3.2220e-05,
          -4.0766e-05, -3.7901e-05, -2.0057e-05, -2.0294e-05, -3.0899e-05,
          -3.3452e-05, -3.0761e-05, -2.3757e-05, -3.0082e-05, -3.7568e-05,
          -3.1157e-05, -3.1568e-05, -3.6838e-05, -2.9389e-05,  1.6219e-06,
           4.6495e-05,  7.3045e-06, -1.0137e-04,  1.6964e-04,  3.4825e-04,
          -5.7342e-04,  1.3340e-04, -3.9608e-02,  1.6935e-01,  7.0987e-01],
         [-2.9762e-04, -7.0939e-06,  2.4640e-04,  2.1438e-04,  1.4383e-04,
           1.1081e-04,  6.3662e-05, -6.9678e-06, -2.1861e-05, -1.2297e-05,
          -2.3371e-05, -4.2028e-05, -5.4611e-05, -6.4683e-05, -7.1668e-05,
          -8.4444e-05, -6.8286e-05, -4.0802e-05, -3.9386e-05, -5.0868e-05,
          -5.8466e-05, -5.4766e-05, -4.1177e-05, -4.0287e-05, -4.4357e-05,
          -4.1906e-05, -